# Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import requests
import json
from dotenv import load_dotenv
import os

# API handling

In [2]:
# loading api key
load_dotenv()
api_key = os.getenv("API_KEY")

In [3]:
#requesting alphavantage url
url = "https://www.alphavantage.co/query?function=DIGITAL_CURRENCY_DAILY&symbol=BTC&market=EUR&apikey= {api_key}" 
response = requests.get(url)
status_code = response.status_code
print(status_code)
data = response.json()
print(json.dumps(data, indent=2))
response.ok

200
{
  "Meta Data": {
    "1. Information": "Daily Prices and Volumes for Digital Currency",
    "2. Digital Currency Code": "BTC",
    "3. Digital Currency Name": "Bitcoin",
    "4. Market Code": "EUR",
    "5. Market Name": "Euro",
    "6. Last Refreshed": "2025-03-26 00:00:00",
    "7. Time Zone": "UTC"
  },
  "Time Series (Digital Currency Daily)": {
    "2025-03-26": {
      "1. open": "81050.89000000",
      "2. high": "81470.40000000",
      "3. low": "80879.26000000",
      "4. close": "81302.93000000",
      "5. volume": "3.42848415"
    },
    "2025-03-25": {
      "1. open": "81014.11000000",
      "2. high": "82067.90000000",
      "3. low": "79916.95000000",
      "4. close": "81040.50000000",
      "5. volume": "339.45558240"
    },
    "2025-03-24": {
      "1. open": "79441.34000000",
      "2. high": "82091.43000000",
      "3. low": "79010.55000000",
      "4. close": "81016.58000000",
      "5. volume": "439.44522481"
    },
    "2025-03-23": {
      "1. open": "774

True

In [4]:
#Extracting the "Time Series (Daily)" part
time_series = data["Time Series (Digital Currency Daily)"]

In [5]:
# Transform into a list of dicts
records = []
for date, values in time_series.items(): # looping through each key-value pair in the "Time Series (Daily)" dictionary
    record = {"date": date}
    record.update({ 
        "open": float(values["1. open"]),
        "high": float(values["2. high"]),
        "low": float(values["3. low"]),
        "close": float(values["4. close"]),
        "volume": float(values["5. volume"])
    })
    records.append(record)

# convert into dataframe
stock_df = pd.DataFrame(records)
stock_df

,date,open,high,low,close,volume
0,2025-03-26,81050.89,81470.40,80879.26,81302.93,3.428484
1,2025-03-25,81014.11,82067.90,79916.95,81040.50,339.455582
2,2025-03-24,79441.34,82091.43,79010.55,81016.58,439.445225
3,2025-03-23,77476.79,79500.00,77463.93,79452.90,138.706858
4,2025-03-22,77674.53,78145.93,77340.05,77503.62,69.495530
...,...,...,...,...,...,...
345,2024-04-15,61870.64,62869.53,58717.47,59747.50,1009.513736
346,2024-04-14,61005.18,62043.20,59021.70,61867.60,760.758338
347,2024-04-13,63330.19,64027.39,57251.00,61007.42,1559.927254
348,2024-04-12,65281.03,66588.53,61300.00,63328.63,984.357623


In [6]:
# converting date column to date time
stock_df["date"] = pd.to_datetime(stock_df["date"])

In [7]:
# converting to csv
stock_df.to_csv("alphavantage_api_btc.csv")

# Saving into SQL 

In [8]:
from sqlalchemy import create_engine

In [9]:
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

# Connect to the default 'postgres' database
conn = psycopg2.connect(
    dbname="stock_data",
    user="postgres",
    password= "",
    host="localhost",
    port="5432"
)


In [10]:
# SQLAlchemy connection to PostgreSQL
engine = create_engine("postgresql+psycopg2://postgres@localhost:5432/stock_data")


In [11]:
stock_df.to_sql('bitcoin_daily_prices', engine, if_exists='replace', index=False)

350

#NOTE - Where is the database located?
Since  using PostgreSQL, your stock_data database is not a standalone file like with SQLite — it's stored inside the PostgreSQL server, which manages everything internally.
So where is it physically located?
On macOS (and in general), PostgreSQL stores its databases in a data directory, managed by the server. This location depends on how you installed PostgreSQ

In [12]:
#checking table was created
df_check = pd.read_sql("SELECT * FROM bitcoin_daily_prices LIMIT 5;", engine)


In [13]:
# checking the schema

query = """
SELECT column_name, data_type, is_nullable
FROM information_schema.columns
WHERE table_name = 'bitcoin_daily_prices';
"""

pd.read_sql(query, engine)


,column_name,data_type,is_nullable
0,date,timestamp without time zone,YES
1,open,double precision,YES
2,high,double precision,YES
3,low,double precision,YES
4,close,double precision,YES
5,volume,double precision,YES


created this table in the terminal: 
CREATE TABLE bitcoin_daily_prices (
    date DATE PRIMARY KEY,
    open DOUBLE PRECISION NOT NULL,
    high DOUBLE PRECISION NOT NULL,
    low DOUBLE PRECISION NOT NULL,
    close DOUBLE PRECISION NOT NULL,
    volume BIGINT NOT NULL
);

In [14]:
#load the data
# stock_df.to_sql('bitcoin_daily_prices', engine, if_exists='append', index=False)

In [15]:

query = """
SELECT *
FROM bitcoin_daily_prices
"""

pd.read_sql(query, engine)

,date,open,high,low,close,volume
0,2025-03-26,81050.89,81470.40,80879.26,81302.93,3.428484
1,2025-03-25,81014.11,82067.90,79916.95,81040.50,339.455582
2,2025-03-24,79441.34,82091.43,79010.55,81016.58,439.445225
3,2025-03-23,77476.79,79500.00,77463.93,79452.90,138.706858
4,2025-03-22,77674.53,78145.93,77340.05,77503.62,69.495530
...,...,...,...,...,...,...
345,2024-04-15,61870.64,62869.53,58717.47,59747.50,1009.513736
346,2024-04-14,61005.18,62043.20,59021.70,61867.60,760.758338
347,2024-04-13,63330.19,64027.39,57251.00,61007.42,1559.927254
348,2024-04-12,65281.03,66588.53,61300.00,63328.63,984.357623
